In [1]:
txt_best_grids = "best_results_svr_cup.txt"
grid_results_name = 'grid_results_svr_cup_v3.csv'

In [2]:
# !pip install ipython-autotime
# !pip install -U scikit-learn
%load_ext autotime

time: 0 ns (started: 2021-01-23 16:13:35 +01:00)


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

time: 0 ns (started: 2021-01-23 16:13:35 +01:00)


In [4]:
# !pwd
# import os
# os.chdir('/content/drive/My Drive/ML/ml-project-master/src/')
# !pwd

time: 0 ns (started: 2021-01-23 16:13:35 +01:00)


In [5]:
import os

os.chdir('../../')
os.getcwd()

'C:\\Users\\Luca\\Desktop\\Elle\\University\\0 - UNIPI\\ML - Machine Learning\\Workspace\\ml-project'

time: 15 ms (started: 2021-01-23 16:13:35 +01:00)


In [6]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

from src.utility import *

time: 5.03 s (started: 2021-01-23 16:13:35 +01:00)


In [7]:
import keras.backend as K


def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

time: 47 ms (started: 2021-01-23 16:13:40 +01:00)


# Import Dataset

In [8]:
def load_cup():
    ml_cup = np.delete(np.genfromtxt('cup/ML-CUP20-TR.csv', 
                                     delimiter=','), obj=0, axis=1)
    return ml_cup[:, :-2], ml_cup[:, -2:]

def load_cup_blind():
    return np.delete(np.genfromtxt('cup/ML-CUP20-TS.csv',
                                   delimiter=','), obj=0, axis=1)
    
def mean_euclidean_error(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    return np.mean(np.linalg.norm(y_pred - y_true, axis=1))

time: 0 ns (started: 2021-01-23 16:13:40 +01:00)


In [9]:
X, y = load_cup()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

time: 31 ms (started: 2021-01-23 16:13:40 +01:00)


In [10]:
print(X.shape)
print(y.shape)

print(X_train.shape)
print(y_train.shape)

(1524, 10)
(1524, 2)
(1219, 10)
(1219, 2)
time: 0 ns (started: 2021-01-23 16:13:41 +01:00)


# Grid Search - Gradient Descent

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer

param_grid = {'estimator__C': [0.1, 1, 10, 100, 1000],
              'estimator__gamma': [0.0001, 0.001, 0.005, 0.01, 0.1, 1, 3, 5],
              'estimator__epsilon': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10], 
              'estimator__kernel': ['rbf', 'linear','sigmoid']} # ['rbf', 'linear','poly','sigmoid']
              

svr = MultiOutputRegressor(SVR())
neg_mean_euclidean_error = make_scorer(mean_euclidean_error, greater_is_better=False)

grid = GridSearchCV(svr, param_grid, scoring=neg_mean_euclidean_error, cv= 3, 
                    verbose=2, n_jobs=6, return_train_score=True)
grid_result = grid.fit(X_train, y_train)


#grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, return_train_score=True, verbose=True)



Fitting 3 folds for each of 1320 candidates, totalling 3960 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    4.8s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   13.2s
[Parallel(n_jobs=6)]: Done 353 tasks      | elapsed:   27.6s
[Parallel(n_jobs=6)]: Done 636 tasks      | elapsed:   43.8s
[Parallel(n_jobs=6)]: Done 1085 tasks      | elapsed:  1.1min
[Parallel(n_jobs=6)]: Done 1572 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 2177 tasks      | elapsed:  2.1min
[Parallel(n_jobs=6)]: Done 2856 tasks      | elapsed:  3.7min
[Parallel(n_jobs=6)]: Done 3629 tasks      | elapsed: 10.3min


time: 12min 50s (started: 2021-01-23 16:13:41 +01:00)


[Parallel(n_jobs=6)]: Done 3960 out of 3960 | elapsed: 12.8min finished


In [12]:
# svr.get_params().keys()

time: 0 ns (started: 2021-01-23 16:26:31 +01:00)


In [13]:
import json

grid_results_name_str = f'Results wrt: {grid_results_name} \n'
grid_params_str = f'Grid on: {json.dumps(grid_result.param_grid)} \n'
grid_results_str = f'Best: {grid_result.best_score_} using {grid_result.best_params_} \n'

with open(txt_best_grids, "a") as file_object:
    file_object.write(grid_results_name_str)
    file_object.write(grid_params_str)
    file_object.write(grid_results_str)
    file_object.write('\n')

time: 0 ns (started: 2021-01-23 16:26:31 +01:00)


In [14]:
print(grid_results_str)

Best: -3.0756991075289624 using {'estimator__C': 10, 'estimator__epsilon': 0.5, 'estimator__gamma': 0.1, 'estimator__kernel': 'rbf'} 

time: 0 ns (started: 2021-01-23 16:26:31 +01:00)


In [15]:
import pandas as pd

# df = pd.DataFrame(grid.cv_results_)

df = pd.DataFrame(grid.cv_results_)[['param_estimator__kernel', 'param_estimator__epsilon', 
                                     'param_estimator__C', 'param_estimator__gamma', 
                                     'mean_test_score', 'std_test_score', 
                                     'mean_train_score', 'std_train_score',
                                     'rank_test_score', 'mean_fit_time']].sort_values(by='rank_test_score')
df.rename(columns={'param_estimator__kernel': 'kernel',
                   'param_estimator__epsilon': 'epsilon',
                   'param_estimator__C': 'C',
                   'param_estimator__gamma': 'gamma',
                   'mean_test_score': 'mean_val_score',
                   'std_test_score':'std_val_score',
                   'rank_test_score': 'rank_val_score'}, inplace=True)
df.mean_train_score *= -1
df.mean_val_score *= -1
df

,kernel,epsilon,C,gamma,mean_val_score,std_val_score,mean_train_score,std_train_score,rank_val_score,mean_fit_time
708,rbf,0.5,10,0.1,3.075699,0.153033,2.427987,0.062120,1,0.141285
732,rbf,1,10,0.1,3.080886,0.161802,2.526474,0.057857,2,0.125745
684,rbf,0.1,10,0.1,3.099244,0.136943,2.372033,0.061290,3,0.163018
660,rbf,0.05,10,0.1,3.105668,0.136960,2.367601,0.060561,4,0.165442
636,rbf,0.01,10,0.1,3.111117,0.137754,2.364853,0.059730,5,0.151657
...,...,...,...,...,...,...,...,...,...,...
1175,sigmoid,0.01,1000,5,87775.810838,1020.246851,88390.661683,2033.280763,1316,0.102729
1151,sigmoid,0.005,1000,5,87775.810854,1020.246804,88390.661739,2033.280793,1317,0.106543
1127,sigmoid,0.001,1000,5,87775.810866,1020.246767,88390.661784,2033.280817,1318,0.099336
1103,sigmoid,0.0005,1000,5,87775.810868,1020.246763,88390.661790,2033.280820,1319,0.104785


time: 31 ms (started: 2021-01-23 16:26:31 +01:00)


In [16]:
df.to_csv(grid_results_name)

time: 16 ms (started: 2021-01-23 16:26:31 +01:00)


In [17]:
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

# import csv

# with open(single_grid_results_filename, mode='w') as grid:
#     grid = csv.writer(grid, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#     for mean, stdev, param in zip(means, stds, params):
#       grid.writerow([mean, stdev, param])

time: 0 ns (started: 2021-01-23 16:26:31 +01:00)


In [18]:
# from sklearn.externals import joblib

# # #save your model or results
# # joblib.dump(grid_result, 'gs_object_monk3.pkl')
# # #load your model for further usage
# # boh = joblib.load("gs_object_monk3.pkl")

# # joblib.dump(grid.best_estimator_, 'gs_best_estimator_monk3.pkl', compress = 1)
# # filename = 'finalized_model.sav'
# # pickle.dump(model, open(filename, 'wb'))

time: 0 ns (started: 2021-01-23 16:26:31 +01:00)
